In [18]:
import pandas as pd
import numpy as np

from influxdb_client import InfluxDBClient, Point, Dialect

import re
import time
import datetime

import warnings
from influxdb_client.client.warnings import MissingPivotFunction

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.colors as colors

import pandasql as ps
import sqlite3

import csv

import warnings 
from influxdb_client.client.warnings import MissingPivotFunction
warnings.simplefilter("ignore", MissingPivotFunction)

import gc

pd.set_option('display.max_rows', 500)

In [19]:
def _parse_line(line):

    rx_dict = {
    'token': re.compile(r'var token = "(?P<token>.*)"\n'),
    'url': re.compile(r'var url = "(?P<url>.*)"\n'),
    'org': re.compile(r'var org = "(?P<org>.*)"\n'),
    'bucket': re.compile(r'var bucket = "(?P<bucket>.*)"\n'),
    }   

    """
    Do a regex search against all defined regexes and
    return the key and match result of the first matching regex

    """
    for key, rx in rx_dict.items():
        match = rx.search(line)
        if match:
            return key, match
    # if there are no matches
    return None, None

filepath = '/root/flexi-pipe/config.go'
# open the file and read through it line by line
with open(filepath, 'r') as file_object:
    line = file_object.readline()
    while line:
        # at each line check for a match with a regex
        key, match = _parse_line(line)

        if key == 'token':
            token = match.group('token')
        elif key == 'url':
            url = match.group('url')
        elif key == 'org':
            org = match.group('org')
        elif key == 'bucket':
            bucket = match.group('bucket')
        
        line = file_object.readline()
# url="http://192.168.20.58:8086"
url = "http://localhost:8086"

In [20]:
start = 1693501608
end = 1694744007

In [21]:
def experiment(start_time, end_time, filepath):
    # Retrieve experiments data from csv
    data = pd.read_csv(filepath, header=None)
    df = pd.DataFrame(data)

    #Rename columns
    experiments = df.rename(columns={0: "start", 1: "end", 2: "topology", 3: "runtime", 4: "parameter", 5: "d", 6: "dlo", 7: "dhi", 8: "dscore", 9: "dlazy", 10: "dout", 11: "gossipFactor", 12: "initialDelay", 13: "interval"}, errors='raise')

    #Correct timestamp
    experiments["start"] = experiments["start"].str.slice(0, 27)
    experiments["end"] = experiments["end"].str.slice(0, 27)

    #String to timestamp
    # experiments['startUnix'] = pd.to_datetime(experiments["start"],format="%Y-%m-%d %H:%M:%S.%f").astype('int64') / 10**9
    # experiments['endUnix'] = pd.to_datetime(experiments["end"],format="%Y-%m-%d %H:%M:%S.%f").astype('int64') / 10**9
    experiments['startUnix'] = pd.to_datetime(experiments["start"],format="mixed", infer_datetime_format=True).astype('int64') / 10**9
    experiments['endUnix'] = pd.to_datetime(experiments["end"],format="mixed", infer_datetime_format=True).astype('int64') / 10**9


    experiments['startUnix'] = pd.to_timedelta(experiments['startUnix'], unit='s').dt.total_seconds().astype(int)#.astype(str)
    experiments['endUnix'] = pd.to_timedelta(experiments['endUnix'], unit='s').dt.total_seconds().astype(int)#.astype(str)

    #Drop fields we don't mneed for the moment
    exp = experiments.drop(columns=["runtime", "initialDelay"]).sort_values(by=["start"])

    #Get times for different intervals
    # intervals = exp["interval"].drop_duplicates().sort_values().reset_index(drop=True)
    # intervals.head(10)

    expTime = exp[exp['startUnix'].astype(int).between(start_time, end_time)]
    # expTime['experiment'] = expTime.index
    expTime = expTime.reset_index().rename({'index':'experiment'}, axis = 'columns')

    return expTime

experiments = experiment(start, end, '../experiments.csv')
experiments.head(300)

/tmp/ipykernel_3527/3663955728.py:16: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  experiments['startUnix'] = pd.to_datetime(experiments["start"],format="mixed", infer_datetime_format=True).astype('int64') / 10**9
/tmp/ipykernel_3527/3663955728.py:17: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  experiments['endUnix'] = pd.to_datetime(experiments["end"],format="mixed", infer_datetime_format=True).astype('int64') / 10**9


,experiment,start,end,topology,parameter,d,dlo,dhi,dscore,dlazy,dout,gossipFactor,interval,startUnix,endUnix
0,300,2023-08-31 17:06:48.9141201,2023-08-31 17:36:48.9166680,unl,reference,8,6,12,4,8,2,0.25,1.00,1693501608,1693503408
1,301,2023-08-31 17:40:21.3151835,2023-08-31 18:10:21.3173764,unl,reference,8,6,12,4,8,2,0.25,1.00,1693503621,1693505421
2,302,2023-08-31 18:13:53.6137805,2023-08-31 18:43:53.6156048,unl,reference,8,6,12,4,8,2,0.25,1.00,1693505633,1693507433
3,303,2023-08-31 18:47:26.3231768,2023-08-31 19:17:26.3252692,unl,reference,8,6,12,4,8,2,0.25,1.00,1693507646,1693509446
4,304,2023-08-31 19:20:59.6306490,2023-08-31 19:50:59.6344278,unl,reference,8,6,12,4,8,2,0.25,1.00,1693509659,1693511459
5,305,2023-08-31 19:54:32.0339718,2023-08-31 20:24:32.0359158,unl,reference,8,6,12,4,8,2,0.25,1.00,1693511672,1693513472
6,306,2023-09-11 18:16:21.3110713,2023-09-11 18:46:21.3522860,unl,reference,8,6,12,4,8,2,0.25,1.00,1694456181,1694457981
7,307,2023-09-11 18:54:26.4001509,2023-09-11 19:24:26.4372422,unl,reference,8,6,12,4,8,2,0.25,1.00,1694458466,1694460266
8,308,2023-09-11 19:31:44.1927688,2023-09-11 20:01:44.2159350,unl,reference,8,6,12,4,8,2,0.25,1.00,1694460704,1694462504
9,309,2023-09-11 20:09:04.8883304,2023-09-11 20:39:04.9399090,unl,reference,8,6,12,4,8,2,0.25,1.00,1694462944,1694464744


In [22]:
def from_influx(url, token, org, measurement, start_time, end_time,grouping_key):
    client = InfluxDBClient(url=url, token=token, org=org,  timeout=900_000)

    # write_api = client.write_api(write_options=SYNCHRONOUS)
    query_api = client.query_api()

    data_frame = query_api.query_data_frame('from(bucket: "gs") '
                                        ' |> range(start: '+str(start_time)+', stop:'+str(end_time)+') '
                                        ' |> filter(fn: (r) => r._measurement == "'+measurement+'") '
                                        ' |> group(columns: ["_measurement", "_field"], mode: "by") '
                                        ' |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")')
    client.close()

    # df = data_frame.drop(columns=['result', 'table','_start', '_stop', '_measurement', 'topic', 'receivedFrom']).sort_values(by=["_time"]).reset_index(drop=True)
    data_frame.reset_index(inplace=True)
    df = data_frame[['_time', grouping_key]].sort_values(by=["_time"]).reset_index(drop=True)
    df["_time"] = pd.to_datetime(df["_time"])

    return df

In [23]:
def from_influx_count(url, token, org, start_time, end_time,grouping_key):
    # start_time = 1693222601 
    # end_time = 1693228430    
    client = InfluxDBClient(url=url, token=token, org=org,  timeout=900_000)
        
        # write_api = client.write_api(write_options=SYNCHRONOUS)
    query_api = client.query_api()
        
    data_frame = query_api.query_data_frame('from(bucket: "gs") '
                       ' |> range(start: '+str(start_time)+', stop:'+str(end_time)+') '
                        '|> filter(fn: (r) => r._measurement == "deliverMessage") '
                        '|> group(columns: ["_measurement", "_field"], mode: "by") '
                        '|> count()')
    client.close()

    if data_frame.empty:
        count = 0
    else:
        count = data_frame["_value"].min().astype(int)
    return count

In [24]:
#Validate data
validate = pd.DataFrame()

for index, row in experiments.iterrows():
    count = from_influx_count(url, token, org, row["startUnix"], row["endUnix"],"_measurement")
    d = {'experiment': [row['experiment']], 'count': [count], 'topology': [row["topology"]], 'd': [row["d"]],'dhi': [row["dhi"]],'dlo': [row["dlo"]],'dlazy': [row["dlazy"]],
        'dscore': [row["dscore"]],'dout': [row["dout"]],'gossipFactor': [row["gossipFactor"]],'interval': [row["interval"]],}
    aux = pd.DataFrame(data=d)
    validate = pd.concat([validate, aux])

# validate.head(200)

exps = experiments.merge(validate, on=['experiment', 'topology','d','dhi','dlo','dlazy','dscore','dout','gossipFactor','interval'])
exps = exps.loc[exps["count"]>0]

exps.head(10)

,experiment,start,end,topology,parameter,d,dlo,dhi,dscore,dlazy,dout,gossipFactor,interval,startUnix,endUnix,count
0,300,2023-08-31 17:06:48.9141201,2023-08-31 17:36:48.9166680,unl,reference,8,6,12,4,8,2,0.25,1.0,1693501608,1693503408,128481
1,301,2023-08-31 17:40:21.3151835,2023-08-31 18:10:21.3173764,unl,reference,8,6,12,4,8,2,0.25,1.0,1693503621,1693505421,215
7,307,2023-09-11 18:54:26.4001509,2023-09-11 19:24:26.4372422,unl,reference,8,6,12,4,8,2,0.25,1.0,1694458466,1694460266,213
9,309,2023-09-11 20:09:04.8883304,2023-09-11 20:39:04.9399090,unl,reference,8,6,12,4,8,2,0.25,1.0,1694462944,1694464744,5570
10,310,2023-09-11 20:46:25.8152460,2023-09-11 21:16:25.8461430,unl,reference,8,6,12,4,8,2,0.25,1.0,1694465185,1694466985,2426
23,323,2023-09-12 04:52:20.2175848,2023-09-12 05:22:20.2610118,unl,4,16,8,20,4,8,2,0.25,0.5,1694494340,1694496140,1411
28,328,2023-09-12 18:00:05.6473282,2023-09-12 18:30:05.6489429,unl,reference,8,6,12,4,8,2,0.25,1.0,1694541605,1694543405,194713
29,329,2023-09-12 18:33:30.3361846,2023-09-12 19:03:30.3400825,unl,reference,8,6,12,4,8,2,0.25,1.0,1694543610,1694545410,243619
30,330,2023-09-12 19:07:00.3403636,2023-09-12 19:37:00.3500787,unl,reference,8,6,12,4,8,2,0.25,1.0,1694545620,1694547420,199084
31,331,2023-09-12 19:40:30.1978792,2023-09-12 20:10:30.2017551,unl,reference,8,6,12,4,8,2,0.25,1.0,1694547630,1694549430,208266


In [25]:
#Remove fault executions

#get median
median = exps.drop(columns=['start','end','startUnix','endUnix', 'parameter', 'experiment']).drop_duplicates()
median = median.groupby(['topology','d','dhi','dlo','dlazy','dscore','dout','gossipFactor','interval']).agg({'count':['median', 'std', 'mean']})
median.columns = median.columns.droplevel(0)
median.reset_index(inplace=True)
median = median.loc[median['median'] > 0]

median.head(50)

,topology,d,dhi,dlo,dlazy,dscore,dout,gossipFactor,interval,median,std,mean
0,general,8,12,6,8,4,2,0.25,1.00,106449.5,7037.234184,104209.166667
1,general,8,16,6,8,4,2,0.25,0.25,3003.0,1874.623251,3599.666667
2,general,8,16,6,8,4,2,0.25,1.00,84943.0,52538.887411,83523.333333
3,general,16,20,8,8,4,2,0.25,0.50,446.0,NaN,446.000000
4,general,16,20,8,8,4,2,0.25,1.00,101127.0,17637.180926,106373.000000
5,unl,8,12,6,8,4,2,0.25,1.00,174168.0,103602.719369,126953.818182
6,unl,8,16,6,8,4,2,0.25,0.25,137051.0,53305.997299,124107.000000
7,unl,8,16,6,8,4,2,0.25,1.00,167606.0,61316.057424,167606.000000
8,unl,10,16,8,8,4,4,0.25,1.00,155548.0,135973.805595,155548.000000
9,unl,16,20,8,8,4,2,0.25,0.50,11650.5,14480.839772,11650.500000


In [26]:
#Validate the data

validation = exps.merge(median, on=['topology','d','dhi','dlo','dlazy','dscore','dout','gossipFactor','interval'])
validated = validation[(validation['count'] >= validation['mean']-(0.15*validation['std']))]
validated = validated.drop(columns=['startUnix', 'endUnix'])
validated.dropna()

validated.head(100)

,experiment,start,end,topology,parameter,d,dlo,dhi,dscore,dlazy,dout,gossipFactor,interval,count,median,std,mean
0,300,2023-08-31 17:06:48.9141201,2023-08-31 17:36:48.9166680,unl,reference,8,6,12,4,8,2,0.25,1.00,128481,174168.0,103602.719369,126953.818182
5,328,2023-09-12 18:00:05.6473282,2023-09-12 18:30:05.6489429,unl,reference,8,6,12,4,8,2,0.25,1.00,194713,174168.0,103602.719369,126953.818182
6,329,2023-09-12 18:33:30.3361846,2023-09-12 19:03:30.3400825,unl,reference,8,6,12,4,8,2,0.25,1.00,243619,174168.0,103602.719369,126953.818182
7,330,2023-09-12 19:07:00.3403636,2023-09-12 19:37:00.3500787,unl,reference,8,6,12,4,8,2,0.25,1.00,199084,174168.0,103602.719369,126953.818182
8,331,2023-09-12 19:40:30.1978792,2023-09-12 20:10:30.2017551,unl,reference,8,6,12,4,8,2,0.25,1.00,208266,174168.0,103602.719369,126953.818182
9,332,2023-09-12 20:14:00.4860767,2023-09-12 20:44:00.4880682,unl,reference,8,6,12,4,8,2,0.25,1.00,174168,174168.0,103602.719369,126953.818182
10,333,2023-09-12 20:47:31.5355445,2023-09-12 21:17:31.5383218,unl,reference,8,6,12,4,8,2,0.25,1.00,239737,174168.0,103602.719369,126953.818182
12,345,2023-09-13 03:29:59.4930994,2023-09-13 03:59:59.4945179,unl,4,16,8,20,4,8,2,0.25,0.50,21890,11650.5,14480.839772,11650.500000
15,336,2023-09-12 22:28:06.9144424,2023-09-12 22:58:06.9194715,unl,1,16,8,20,4,8,2,0.25,1.00,152703,133797.0,14156.425290,137166.000000
17,338,2023-09-12 23:35:11.7401999,2023-09-13 00:05:11.7436074,unl,2,8,6,16,4,8,2,0.25,1.00,210963,167606.0,61316.057424,167606.000000


In [27]:
final = validated.drop(columns=['experiment', 'start','end', 'parameter', 'count', 'std', 'median', 'mean']).drop_duplicates()
final['n_nodes'] = 23
final = final.reset_index().rename({'index':'identifier'}, axis = 'columns')
final.head(10)

,identifier,topology,d,dlo,dhi,dscore,dlazy,dout,gossipFactor,interval,n_nodes
0,0,unl,8,6,12,4,8,2,0.25,1.00,23
1,12,unl,16,8,20,4,8,2,0.25,0.50,23
2,15,unl,16,8,20,4,8,2,0.25,1.00,23
3,17,unl,8,6,16,4,8,2,0.25,1.00,23
4,19,unl,8,6,16,4,8,2,0.25,0.25,23
5,22,unl,16,8,20,6,8,2,0.25,1.00,23
6,24,unl,10,8,16,4,8,4,0.25,1.00,23
7,26,general,8,6,12,4,8,2,0.25,1.00,23
8,33,general,16,8,20,4,8,2,0.25,1.00,23
9,35,general,8,6,16,4,8,2,0.25,1.00,23


In [28]:
#Limit extraction to 15 minutes

experiments = validated.drop(columns=['count','median', 'mean','std']).rename(columns={'start':'originalStart','end':'originalEnd'})

experiments['originalStart'] = pd.to_datetime(experiments["originalStart"], format='mixed')
experiments['originalEnd'] = pd.to_datetime(experiments["originalEnd"], format='mixed')

experiments['start'] = experiments['originalStart'] + pd.Timedelta(hours=0, minutes=7, seconds=30)
experiments['end'] = experiments['originalEnd'] - pd.Timedelta(hours=0, minutes=7, seconds=30)

experiments['startUnix'] = pd.to_datetime(experiments["start"],format="mixed").astype('int64') / 10**9
experiments['endUnix'] = pd.to_datetime(experiments["end"],format="mixed").astype('int64') / 10**9

experiments['startUnix'] = pd.to_timedelta(experiments['startUnix'], unit='s').dt.total_seconds().astype(int)#.astype(str)
experiments['endUnix'] = pd.to_timedelta(experiments['endUnix'], unit='s').dt.total_seconds().astype(int)#.astype(str)

experiments.to_csv('experiments_filtered.csv')
experiments.head(10)

,experiment,originalStart,originalEnd,topology,parameter,d,dlo,dhi,dscore,dlazy,dout,gossipFactor,interval,start,end,startUnix,endUnix
0,300,2023-08-31 17:06:48.914120100,2023-08-31 17:36:48.916668000,unl,reference,8,6,12,4,8,2,0.25,1.0,2023-08-31 17:14:18.914120100,2023-08-31 17:29:18.916668000,1693502058,1693502958
5,328,2023-09-12 18:00:05.647328200,2023-09-12 18:30:05.648942900,unl,reference,8,6,12,4,8,2,0.25,1.0,2023-09-12 18:07:35.647328200,2023-09-12 18:22:35.648942900,1694542055,1694542955
6,329,2023-09-12 18:33:30.336184600,2023-09-12 19:03:30.340082500,unl,reference,8,6,12,4,8,2,0.25,1.0,2023-09-12 18:41:00.336184600,2023-09-12 18:56:00.340082500,1694544060,1694544960
7,330,2023-09-12 19:07:00.340363600,2023-09-12 19:37:00.350078700,unl,reference,8,6,12,4,8,2,0.25,1.0,2023-09-12 19:14:30.340363600,2023-09-12 19:29:30.350078700,1694546070,1694546970
8,331,2023-09-12 19:40:30.197879200,2023-09-12 20:10:30.201755100,unl,reference,8,6,12,4,8,2,0.25,1.0,2023-09-12 19:48:00.197879200,2023-09-12 20:03:00.201755100,1694548080,1694548980
9,332,2023-09-12 20:14:00.486076700,2023-09-12 20:44:00.488068200,unl,reference,8,6,12,4,8,2,0.25,1.0,2023-09-12 20:21:30.486076700,2023-09-12 20:36:30.488068200,1694550090,1694550990
10,333,2023-09-12 20:47:31.535544500,2023-09-12 21:17:31.538321800,unl,reference,8,6,12,4,8,2,0.25,1.0,2023-09-12 20:55:01.535544500,2023-09-12 21:10:01.538321800,1694552101,1694553001
12,345,2023-09-13 03:29:59.493099400,2023-09-13 03:59:59.494517900,unl,4,16,8,20,4,8,2,0.25,0.5,2023-09-13 03:37:29.493099400,2023-09-13 03:52:29.494517900,1694576249,1694577149
15,336,2023-09-12 22:28:06.914442400,2023-09-12 22:58:06.919471500,unl,1,16,8,20,4,8,2,0.25,1.0,2023-09-12 22:35:36.914442400,2023-09-12 22:50:36.919471500,1694558136,1694559036
17,338,2023-09-12 23:35:11.740199900,2023-09-13 00:05:11.743607400,unl,2,8,6,16,4,8,2,0.25,1.0,2023-09-12 23:42:41.740199900,2023-09-12 23:57:41.743607400,1694562161,1694563061


In [12]:
def from_influx_message(url, token, org, start_time, end_time):
    client = InfluxDBClient(url=url, token=token, org=org,  timeout=900_000)

    # write_api = client.write_api(write_options=SYNCHRONOUS)
    query_api = client.query_api()

    data_frame = query_api.query_data_frame('from(bucket: "gs") '
                                        ' |> range(start: '+str(start_time)+', stop:'+str(end_time)+') '
                                        ' |> filter(fn: (r) => r._measurement == "message" and r._field == "type" and (r._value == 11 or r._value == 12 or r._value == 3 or r._value == 2)) '
                                        ' |> group(columns: ["_measurement", "_field"], mode: "by") '
                                        ' |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")')
    client.close()

    # df = data_frame.drop(columns=['result', 'table','_start', '_stop', '_measurement', 'topic', 'receivedFrom']).sort_values(by=["_time"]).reset_index(drop=True)
    data_frame.reset_index(inplace=True)
    df = data_frame[['_time', 'type']].sort_values(by=["_time"]).reset_index(drop=True)
    df["_time"] = pd.to_datetime(df["_time"])

    return df

In [13]:
message = pd.DataFrame()
execs = experiments.drop(columns=['originalStart', 'originalEnd'])
# final=final.loc[final["identifier"] == 1]

for index, row in final.iterrows():
    # print(row["start"], row["end"], row["experiment"])
    execs = exps.loc[(exps["topology"] == row["topology"]) & (exps["d"] == row["d"]) &
                            (exps["dlo"] == row["dlo"]) & (exps["dhi"] == row["dhi"]) &
                            (exps["dscore"] == row["dscore"]) & (exps["dlazy"] == row["dlazy"]) &
                            (exps["dout"] == row["dout"]) & (exps["gossipFactor"] == row["gossipFactor"]) &                    
                            (exps["interval"] == row["interval"])]

    for idx, exec in execs.iterrows():
        # print(exec["startUnix"], exec["endUnix"])
        query_mess = from_influx_message(url, token, org, exec["startUnix"], exec["endUnix"])

        query_mess["identifier"] = row["identifier"]

        query_mess["experiment"] = exec["experiment"]

        query_mess["start"] = exec["start"]
        query_mess["end"] = exec["end"]
            
        query_mess["_time"] = pd.to_datetime(query_mess["_time"])
        message = pd.concat([message,query_mess])

        query_mess.to_csv('./mess_'+str(exec["experiment"])+'.csv')
        
        del query_mess
        # del deliverMessage
        gc.collect()

message.head(10)

,_time,type,identifier,experiment,start,end
0,2023-08-25 17:04:39.427790+00:00,11.0,1,152,2023-08-25 16:50:23.6899992,2023-08-25 17:20:23.6921991
1,2023-08-25 17:04:39.427797+00:00,11.0,1,152,2023-08-25 16:50:23.6899992,2023-08-25 17:20:23.6921991
2,2023-08-25 17:04:39.427811+00:00,11.0,1,152,2023-08-25 16:50:23.6899992,2023-08-25 17:20:23.6921991
3,2023-08-25 17:04:39.427817+00:00,11.0,1,152,2023-08-25 16:50:23.6899992,2023-08-25 17:20:23.6921991
4,2023-08-25 17:04:39.427821+00:00,11.0,1,152,2023-08-25 16:50:23.6899992,2023-08-25 17:20:23.6921991
5,2023-08-25 17:04:39.427825+00:00,11.0,1,152,2023-08-25 16:50:23.6899992,2023-08-25 17:20:23.6921991
6,2023-08-25 17:04:39.427830+00:00,11.0,1,152,2023-08-25 16:50:23.6899992,2023-08-25 17:20:23.6921991
7,2023-08-25 17:04:39.427834+00:00,11.0,1,152,2023-08-25 16:50:23.6899992,2023-08-25 17:20:23.6921991
8,2023-08-25 17:04:39.427845+00:00,11.0,1,152,2023-08-25 16:50:23.6899992,2023-08-25 17:20:23.6921991
9,2023-08-25 17:04:39.427860+00:00,11.0,1,152,2023-08-25 16:50:23.6899992,2023-08-25 17:20:23.6921991


In [14]:
message.to_csv('./messageTimeSeries.csv')

In [15]:
del message
gc.collect()

0

In [41]:
def from_influx(url, token, org, measurement, start_time, end_time,grouping_key):
    client = InfluxDBClient(url=url, token=token, org=org,  timeout=900_000)

    # write_api = client.write_api(write_options=SYNCHRONOUS)
    query_api = client.query_api()

    data_frame = query_api.query_data_frame('from(bucket: "gs") '
                                        ' |> range(start: '+str(start_time)+', stop:'+str(end_time)+') '
                                        ' |> filter(fn: (r) => r._measurement == "'+measurement+'") '
                                        ' |> group(columns: ["_measurement", "_field"], mode: "by") '
                                        ' |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")')
    client.close()

    # df = data_frame.drop(columns=['result', 'table','_start', '_stop', '_measurement', 'topic', 'receivedFrom']).sort_values(by=["_time"]).reset_index(drop=True)
    if data_frame.empty == False:
        data_frame.reset_index(inplace=True)
        df = data_frame[['_time', grouping_key]].sort_values(by=["_time"]).reset_index(drop=True)
        df["_time"] = pd.to_datetime(df["_time"])
    else:
        df = data_frame

    return df

In [42]:
gossip = pd.DataFrame()
execs = experiments.drop(columns=['originalStart', 'originalEnd'])

for index, row in final.iterrows():
    # print(row["start"], row["end"], row["experiment"])
    execs = exps.loc[(exps["topology"] == row["topology"]) & (exps["d"] == row["d"]) &
                            (exps["dlo"] == row["dlo"]) & (exps["dhi"] == row["dhi"]) &
                            (exps["dscore"] == row["dscore"]) & (exps["dlazy"] == row["dlazy"]) &
                            (exps["dout"] == row["dout"]) & (exps["gossipFactor"] == row["gossipFactor"]) &                    
                            (exps["interval"] == row["interval"])]

    for idx, exec in execs.iterrows():
        # print(exec["startUnix"], exec["endUnix"])
        query_iwant = from_influx(url, token, org, "iwant", exec["startUnix"], exec["endUnix"],"_measurement")

        if query_iwant.empty == False:
            query_iwant["identifier"] = row["identifier"]
            query_iwant["experiment"] = exec["experiment"]
 
            query_iwant["_time"] = pd.to_datetime(query_iwant["_time"])
            gossip = pd.concat([gossip, query_iwant])

            query_iwant.to_csv('./iwant_'+str(exec["experiment"])+'.csv')

            del query_iwant

        query_ihave = from_influx(url, token, org, "ihave", exec["startUnix"], exec["endUnix"],"_measurement")
        if query_ihave.empty == False:
            query_ihave["identifier"] = row["identifier"]
            query_ihave["experiment"] = exec["experiment"]

            query_ihave["_time"] = pd.to_datetime(query_ihave["_time"])
            gossip = pd.concat([gossip, query_ihave])

            query_ihave.to_csv('./ihave_'+str(exec["experiment"])+'.csv')
        
            del query_ihave

        gc.collect()

gossip.head(10)

,_time,_measurement,identifier,experiment
0,2023-08-25 17:04:39.559375+00:00,iwant,1,152
1,2023-08-25 17:04:39.559955+00:00,iwant,1,152
2,2023-08-25 17:04:39.560356+00:00,iwant,1,152
3,2023-08-25 17:04:39.574611+00:00,iwant,1,152
4,2023-08-25 17:04:39.575129+00:00,iwant,1,152
5,2023-08-25 17:04:39.575320+00:00,iwant,1,152
6,2023-08-25 17:04:39.575510+00:00,iwant,1,152
7,2023-08-25 17:04:39.585263+00:00,iwant,1,152
8,2023-08-25 17:04:39.585524+00:00,iwant,1,152
9,2023-08-25 17:04:39.585600+00:00,iwant,1,152


In [43]:
gossip.to_csv('./gossipTimeSeries.csv')